# Processing dataset -- Chunking

In [ ]:
import lzma
import json
import pandas
from chunkipy import TextChunker, TokenEstimator
import os

In [ ]:
file_path = "data/train.doj_guidance.jsonl.xz"

docs = []
with lzma.open(file_path, "rt", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 100:
            break
        docs.append(json.loads(line))

print(f"Loaded {len(docs)} documents.")
print("First document preview:", docs[0])

In [ ]:
# Print the keys (fields) of the first document
print(docs[0].keys())

In [ ]:
def chunk_and_save(text, chunk_size, tokens, overlap_percent, docId, output_dir="resources/data/chunks/guidance"):
    """
    Splits the input text into overlapping chunks and saves each chunk as a JSON file.

    Args:
        text (str): The text to be chunked.
        chunk_size (int): The size of each chunk.
        tokens (bool): Whether to chunk by tokens.
        overlap_percent (float): Percentage of overlap between chunks.
        document_name (str): Name of the original document.
        output_dir (str): Directory to save the chunk files.
    """
    os.makedirs(output_dir, exist_ok=True)
    text_chunker = TextChunker(chunk_size, tokens=tokens, overlap_percent=overlap_percent)
    chunks = text_chunker.chunk(text)
    for i, chunk in enumerate(chunks):
        chunk_data = {
            "document_category": "guidance", #change here
            "docId": docId,
            "chunk_index": i + 1,
            "chunk_text": chunk
        }
        chunk_filename = f"{docId}_chunk_{i + 1}.json"
        chunk_path = os.path.join(output_dir, chunk_filename)
        with open(chunk_path, "w", encoding="utf-8") as f:
            json.dump(chunk_data, f, ensure_ascii=False, indent=2)
        print(f"Saved {chunk_filename} ({len(chunk)} chunk characters)")

In [ ]:
#loop over docs
for idx, doc in enumerate(docs):
    url = doc['url']
    docId = "guidance_"+str(idx)
    chunk_and_save(doc['text'], chunk_size=500, tokens=True, overlap_percent=0.1, docId=docId, output_dir= "resources/data/chunks/guidance")
    print("--------------")

## Query generation

In [ ]:
# TODO

## Dataset creation

In [ ]:

import os
import json
import pandas as pd

base_Folder = "resources/data/querygen"
filenames = next(os.walk(base_Folder), (None, None, []))[2]  # [] if no file


chunks = []
queries = []
labels = []
query_count = 0
for filename in filenames:
    with open(os.path.join(base_Folder, filename), "r") as file:
        chunks_batch = json.load(file)
        for chunk in chunks_batch:
            labels_chunk = []
            chunk_queries = [q for q, s in zip(chunk["querygen"], chunk["querygen_score"]) if float(s) >= 1.5]
            for chunk_query in chunk_queries:
                queries.append({"query_id": query_count, "query": chunk_query})
                labels_chunk.append({"query_id": query_count, 
                           "doc_id": chunk["doc_id"],  
                           "chunk_id": chunk["chunk_id"],
                           "label": "Relevant"})
                query_count += 1
            del chunk["querygen"]
            del chunk["querygen_score"]
            chunks.append(chunk)
            labels.extend(labels_chunk)
            
            
documents_df = pd.DataFrame(chunks)
documents_df.rename(columns={"document_category": "category", "text": "content"}, inplace=True)
documents_df.to_csv("dataset/legal-docs/document.csv", index=False, encoding="utf-8")


queries_df = pd.DataFrame(queries)
queries_df.set_index('query_id', inplace=True)
queries_df.to_csv("dataset/legal-docs/query.csv", index=True, encoding="utf-8")


labels_df = pd.DataFrame(labels)
labels_df.to_csv("dataset/legal-docs/positive-label.csv", index=False, encoding="utf-8")        

In [ ]:
documents_df = pd.read_csv("dataset/legal-docs/document.csv")
queries_df = pd.read_csv("dataset/legal-docs/query.csv")
positive_labels_df = pd.read_csv("dataset/legal-docs/positive-label.csv")


In [ ]:
documents_df.head()

In [ ]:
positive_labels_df.head()

In [ ]:
positive_labels_df[["doc_id","chunk_id"]].value_counts()

In [ ]:
queries_df.head()

In [ ]:
# create negative labels

import random
from pyterrier_doc2query import QueryScorer
from pyterrier_dr import ElectraScorer

def create_negative_labels(labels_df, documents_df, quires_df, query_scorer):

    query_id = labels_df["query_id"].tolist()
    random.shuffle(query_id)

    # build df for Query Scorer
    query_list = [{
        "query_id": query_id[i],
        "doc_id": labels_df['doc_id'][i],
        "chunk_id": labels_df['chunk_id'][i],
        "text": documents_df[documents_df["id"] == f"{labels_df['doc_id'][i]}__{labels_df['chunk_id'][i]}"]["content"].values[0],
        "querygen": quires_df[quires_df["query_id"] == query_id[i]]["query"].values[0]
    }
    for i in range(len(labels_df))
    ]

    scores_df = query_scorer.transform(pd.DataFrame.from_records(query_list))

    return scores_df


In [ ]:
n_runs = 1

negative_labels_df = pd.DataFrame()

for _ in range(n_runs):
     negative_labels_df = pd.concat((negative_labels_df,
                                     create_negative_labels(positive_labels_df, documents_df, queries_df, QueryScorer(ElectraScorer()))
                                     )
                                    )
negative_labels_df.reset_index(inplace=True)

In [ ]:
negative_labels_df

In [ ]:
# create relevance
relevance = []
for i in range(len(negative_labels_df)):
    if negative_labels_df['querygen_score'][i][0] > 1.5:
        relevance.append("Relevant")
    else:
        relevance.append("Irrelevant")


negative_labels_df["label"] = relevance
negative_labels_df = negative_labels_df[positive_labels_df.columns]

In [ ]:
# concat with positive labels
labels_df = pd.concat((positive_labels_df, negative_labels_df))
labels_df.reset_index(inplace=True)

In [ ]:
labels_df

In [ ]:
labels_df.to_csv("dataset/legal-docs/label.csv", index=False, encoding="utf-8") 